In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12-volume/sample data.csv", header=True, inferSchema=True)
display(df)

In [0]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/default/batch12-volume/Order_Excel.csv")
#display(df) # to display in tabular format
#df.show() #By default, it shows only 20 rows
df.printSchema #to print the schema

In [0]:
df = spark.read.csv("/Volumes/workspace/default/batch12-volume/Order_Excel.csv", header=True, inferSchema=True)
display(df)

#Define schema progarmatically

In [0]:
#Define schema progarmatically
from pyspark.sql.types import *
orderSchema = StructType([StructField("Region", StringType() ,True)
,StructField("Country", StringType() ,True)
,StructField("ItemType", StringType() ,True)
,StructField("SalesChannel", StringType() ,True)
,StructField("OrderPriority", StringType() ,True)
,StructField("OrderID", IntegerType() ,True)
,StructField("UnitsSold", IntegerType() ,True)
,StructField("UnitPrice", DoubleType() ,True)
,StructField("UnitCost", DoubleType() ,True)
,StructField("TotalRevenue", DoubleType() ,True)
,StructField("TotalCost", DoubleType() ,True)
,StructField("TotalProfit", DoubleType() ,True)
])


df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
df.printSchema()
df.schema

# Define schema Declaratively wayb(easy way 😅💪😍)

In [0]:
orderSchema = 'Region String ,Country String ,ItemType String ,SalesChannel String ,OrderPriority String ,OrderID Integer ,UnitsSold Integer ,UnitPrice Double ,UnitCost Double ,TotalRevenue Double ,TotalCost Double ,TotalProfit Double'
                          
df = spark.read.load("/Volumes/workspace/default/batch12-volume/Order_Excel.csv",format="csv", header=True, schema=orderSchema)
df.printSchema()
#df.schema
#display(df)

# HOW to pick particular coulmn using select function 

In [0]:
df1 = df.select("Region","Country","SalesChannel")
display(df1)

In [0]:
#Alternative ways for selecting the particular or specfic column

from pyspark.sql.functions import *
df1 = df.select("Region",col("Country"), column("SalesChannel"), df["SalesChannel"], df.TotalCost)
display(df1)


# Alias and EXPR 

In [0]:
#Alias
from pyspark.sql.functions import *
df1 = df.select("Region",col("Country").alias("New_country"), column("SalesChannel"), df["SalesChannel"], df.TotalCost, expr("TotalProfit*100/TotalCost").alias("profit%"))
display(df1)

In [0]:
from pyspark.sql.functions import *
#Add column
#Rename column
#Delete the column
df4 = df.withColumn("StoreName", lit("Mumbai-Dmart"))
display(df4)

In [0]:
df5 = df.withColumn("ProfitPercent", expr("TotalProfit*100/TotalCost"))
display(df5)

# To creat SQLview

In [0]:
df.createOrReplaceTempView("orders")
df8 = spark.sql("select Region, Country from orders where Country ='India' ")
display(df8)

In [0]:
%sql
select * from orders;

In [0]:
#You have to load a orders data. Now find out the following.
# 1. Display all the country where Totalprofit is more than 10000


df_profit = df.filter(col("TotalProfit") > 10000)
df_Count = df_profit.select(col("Country") == "India"))
display(df_Count)

In [0]:
#2) Display all the Orders details where Region is Asia and Country is India

df_order_detailes = df.filter((col("Region") == "Asia") & (col("Country") == "India"))
display(df_order_detailes)